In [1]:
"""
Using python plotly, to create Bar chart and/or Line chart (hereafter called B_chart and L_chart  ) by retrieving the data called  “funds_daily_2015_to_2036.csv” (this is the file created and named by another python file for the following:

When inputing an action called “Your_return”, yoo will asked to Input the type of Fund (i.e. A,B,C,or D) and  a start_date and a current_date, plot a L_chart to reflect the chosen fund value change between these two dates, and beside the chart showing the percentage change from start_date to current_date. Output of this chart should be in separate files

Similar to above, When inputing an action called “Past_return”, and after Inputing the type of Fund (i.e. A,B,C,or D) , input an end_date, and further to input a start_date with choices limited to and named as “Past_3_months”, “Past_12_months”, “Past_36_months”, “Past_60_months”, “Past_since_2016”, “Past_since_this_year”, which respectively setting the start_date to “91 days early than the current_date”,   “183 days early than the end_date”, “365 days early than the end_date”,   “1095 days early than the end_date”, “1826 days early than the end_date”,   “exactly on 2015.12.31” , “on January 1 of a year equal to that of current year”, afterwards create a L_chart and beside the chart showing  the percentage change from start_date to current_date. Output of the six charts should be in separate files.

Similar to above, When inputing an action called “Illustrate_return”, adopting the “Past_return” mode  and  let the current_date be 2025.12.31 and running “Past_since_2016”, “Past_since_this_year” for all of the four funds to print out  L_chart.  In addition, a L-Chart to putting all funds together in one L_chart  running “Past_since_2016” as well as a L-Chart to putting all funds together in one L_chart  running “Past_since_this_year”. Beside the chart showing  the percentage change from start_date to current_date. Output for the 9 charts (2 x 4   +1 =9) should be in separate files.  Moreover , create four B_chart (Barchart) to pour all four funds together the illustrate the fund performance (i.e. the percentage change from start_date to current_date. ) during “Past_since_2016”, and“Past_since_this_year”.

Similar to above, When inputing an action called “Illustrate_B_return”, adopting the “Past_return” mode  and  let the current_date be 2025.12.31 and running “Past_since_2016”, “Past_since_this_year”, for all of the four funds to print out  L_chart.  In addition, aputting all funds together in a L_chart as well as a B_chart respectively  running “Past_since_this_year”, “Past_since_2023”,  “Past_since_ 2020” and “Past_since_2016".  Beside the chart showing  the percentage change from start_date to current_date. Output in separate files.  

The x-axis for the L-chart should only show year, but if the days between the start_date and current_date is less than 366 days, the x-axis can show year and Mar, Jun, Sep, Dec ( which mean March, June, September, December.

In summary, Inputs by user include: (1) Please choose your action: “Your_return” or “Past-return”,  “illustrate_return” (2) Please choose a fund: A,B,C or D , (3) Please input a current_date,  (4a) upon “Your return” request, please input a star_date, (4b) upon “Past_return” request, please input one of the six:  “Past_3_months”, “Past_12_months”, “Past_36_months”, “Past_60_months”, “Past_since_2016”, “Past_since_this_year”.

"""
import pandas as pd
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go

# ───────────────────────────────────────────────────────────────
# IMPROVED DATA LOADING
# ───────────────────────────────────────────────────────────────

print("=== Loading and validating data ===")

try:
    df = pd.read_csv("funds_daily_2015_to_2036.csv")
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors='coerce')
    df['Date'] = df['Date'].dt.tz_localize(None)
    df = df.dropna(subset=['Date'])
    df = df.set_index('Date').sort_index()
    
    print("→ Loaded successfully!")
    print("Shape:", df.shape)
    print("Date range:", df.index.min().date(), "→", df.index.max().date())
    
    key_dates = ['2015-12-31', '2025-12-31', '2036-12-31']
    print("\nKey date availability:")
    for d in key_dates:
        dt = pd.to_datetime(d)
        if dt in df.index:
            row = df.loc[dt]
            print(f"   {d}: Found → A={row['Fund_A']:.2f}  B={row['Fund_B']:.2f}  C={row['Fund_C']:.2f}  D={row['Fund_D']:.2f}")
        else:
            print(f"   {d}: NOT FOUND")
    
except Exception as e:
    print("Error loading CSV:")
    print(e)
    exit()

print("\nData ready. Starting interactive mode...\n")

# ───────────────────────────────────────────────────────────────
# FUND COLUMN MAPPING
# ───────────────────────────────────────────────────────────────

fund_map = {'A': 'Fund_A', 'B': 'Fund_B', 'C': 'Fund_C', 'D': 'Fund_D'}

# ───────────────────────────────────────────────────────────────
# HELPER FUNCTIONS
# ───────────────────────────────────────────────────────────────

def calculate_pct_change(start_val, end_val):
    return ((end_val - start_val) / start_val) * 100 if start_val != 0 else 0

def get_start_date(current_date, choice):
    if choice == "Past_3_months":
        return current_date - timedelta(days=91)
    elif choice == "Past_12_months":
        return current_date - timedelta(days=365)
    elif choice == "Past_36_months":
        return current_date - timedelta(days=1095)
    elif choice == "Past_60_months":
        return current_date - timedelta(days=1826)
    elif choice == "Past_since_2016":
        return datetime(2015, 12, 31)
    elif choice == "Past_since_this_year":
        return datetime(current_date.year, 1, 1)
    else:
        raise ValueError("Invalid choice for start_date")

def create_l_chart(data, fund_cols, title, start_date, current_date, is_combined=False):
    fig = go.Figure()
    
    pct_text = ''
    
    if is_combined:
        # Single shared y-axis for all funds (A/B/C/D on same scale)
        for col in fund_cols:
            fig.add_trace(go.Scatter(
                x=data.index, 
                y=data[col], 
                mode='lines', 
                name=col
            ))
        
        pct_text = '<br>'.join([f"{col}: {calculate_pct_change(data[col].iloc[0], data[col].iloc[-1]):.2f}%" for col in fund_cols])
        
        fig.update_layout(
            title=title,
            xaxis_title='Year',
            yaxis_title='Value',
            yaxis=dict(range=[0, None]),  # Shared scale from 0
            legend=dict(x=0, y=1.1, orientation='h')
        )
    else:
        col = fund_cols[0]
        fig.add_trace(go.Scatter(x=data.index, y=data[col], mode='lines', name=col))
        pct_change = calculate_pct_change(data[col].iloc[0], data[col].iloc[-1])
        pct_text = f"Percentage Change: {pct_change:.2f}%"
        
        # Y-axis logic
        if col == 'Fund_D':
            min_val = data[col].min()
            max_val = data[col].max()
            buffer = (max_val - min_val) * 0.05
            y_min = max(80, min_val * 0.98)
            y_max = max_val + buffer
            fig.update_layout(
                title=title,
                xaxis_title='Year',
                yaxis_title='Value',
                yaxis=dict(range=[y_min, y_max])
            )
        else:
            fig.update_layout(
                title=title,
                xaxis_title='Year',
                yaxis_title='Value',
                yaxis=dict(range=[0, None])
            )
    
    # X-axis formatting & visibility
    days_diff = (current_date - start_date).days
    if days_diff > 365:
        fig.update_xaxes(dtick='M12', tickformat='%Y', showline=True, linewidth=1.5, linecolor='black', gridcolor='lightgray')
    else:
        fig.update_xaxes(dtick='M3', tickformat='%Y %b', showline=True, linewidth=1.5, linecolor='black', gridcolor='lightgray')
    
    fig.update_xaxes(zeroline=False)
    
    fig.add_annotation(
        x=1.05, y=0.5, text=pct_text, showarrow=False,
        xref='paper', yref='paper', align='left', font=dict(size=12)
    )
    
    fig.update_layout(width=900, height=600, margin=dict(r=220))
    
    return fig

def create_b_chart(pct_changes, title):
    bar_df = pd.DataFrame({'Fund': list(pct_changes.keys()), 'Percentage Change': list(pct_changes.values())})
    fig = px.bar(bar_df, x='Fund', y='Percentage Change', title=title)
    fig.update_yaxes(range=[0, None])
    return fig

# ───────────────────────────────────────────────────────────────
# ACTION FUNCTIONS
# ───────────────────────────────────────────────────────────────

def your_return(fund, start_date_str, current_date_str):
    start_date = pd.to_datetime(start_date_str)
    current_date = pd.to_datetime(current_date_str)
    fund_col = fund_map[fund.upper()]
    data = df.loc[start_date:current_date, [fund_col]]
    title = f"{fund_col} Value Change from {start_date_str} to {current_date_str}"
    fig = create_l_chart(data, [fund_col], title, start_date, current_date)
    filename = f"{fund_col}_Your_return_{start_date_str}_to_{current_date_str}.html"
    fig.write_html(filename, include_plotlyjs='cdn', auto_open=False)
    print(f"Chart saved to {filename}")

def past_return(fund, end_date_str, choice):
    end_date = pd.to_datetime(end_date_str)
    start_date = get_start_date(end_date, choice)
    fund_col = fund_map[fund.upper()]
    data = df.loc[start_date:end_date, [fund_col]]
    title = f"{fund_col} {choice} to {end_date_str}"
    fig = create_l_chart(data, [fund_col], title, start_date, end_date)
    filename = f"{fund_col}_{choice}_to_{end_date_str}.html"
    fig.write_html(filename, include_plotlyjs='cdn', auto_open=False)
    print(f"Chart saved to {filename}")

def illustrate_return():
    current_date = datetime(2025, 12, 31)
    current_date_str = "2025-12-31"
    choices = ["Past_since_2016", "Past_since_this_year"]
    funds = ["A", "B", "C", "D"]
    
    # Individual charts (only original two choices)
    for choice in choices:
        start_date = get_start_date(current_date, choice)
        data = df.loc[start_date:current_date]
        for fund in funds:
            fund_col = fund_map[fund]
            title = f"{fund_col} {choice} to {current_date_str}"
            fig = create_l_chart(data, [fund_col], title, start_date, current_date)
            filename = f"{fund_col}_{choice}_to_{current_date_str}.html"
            fig.write_html(filename, include_plotlyjs='cdn', auto_open=False)
            print(f"Chart saved to {filename}")
    
    # Combined L-charts and Bar charts - four fixed periods
    combined_periods = [
        ("Past_since_2016", datetime(2016, 1, 1), "Since_2016-01-01"),
        ("Since_2020-01-01", datetime(2020, 1, 1), "Since_2020-01-01"),
        ("Since_2023-01-01", datetime(2023, 1, 1), "Since_2023-01-01"),
        ("Past_since_this_year", datetime(2025, 1, 1), "Since_this_year")
    ]
    
    for _, start_date, title_suffix in combined_periods:
        data = df.loc[start_date:current_date]
        title = f"All Funds {title_suffix} to {current_date_str}"
        fig = create_l_chart(data, ['Fund_A', 'Fund_B', 'Fund_C', 'Fund_D'], title, start_date, current_date, is_combined=True)
        filename = f"Combined_All_Funds_{title_suffix}_to_{current_date_str}.html"
        fig.write_html(filename, include_plotlyjs='cdn', auto_open=False)
        print(f"Chart saved to {filename}")
    
    # Bar charts for the same four periods
    for _, start_date, title_suffix in combined_periods:
        data = df.loc[start_date:current_date]
        pct_changes = {}
        for fund in funds:
            fund_col = fund_map[fund]
            pct_changes[fund_col] = calculate_pct_change(data[fund_col].iloc[0], data[fund_col].iloc[-1])
        title = f"Bar Chart % Change All Funds {title_suffix} to {current_date_str}"
        fig = create_b_chart(pct_changes, title)
        filename = f"B_chart_All_Funds_{title_suffix}_to_{current_date_str}.html"
        fig.write_html(filename, include_plotlyjs='cdn', auto_open=False)
        print(f"Chart saved to {filename}")

def illustrate_b_return():
    current_date = datetime(2025, 12, 31)
    current_date_str = "2025-12-31"
    choices = ["Past_since_2016", "Past_since_this_year", "Past_36_months", "Past_60_months"]
    funds = ["A", "B", "C", "D"]
    
    for choice in choices:
        start_date = get_start_date(current_date, choice)
        data = df.loc[start_date:current_date]
        for fund in funds:
            fund_col = fund_map[fund]
            title = f"{fund_col} Illustrate_B {choice} to {current_date_str}"
            fig = create_l_chart(data, [fund_col], title, start_date, current_date)
            filename = f"{fund_col}_Illustrate_B_{choice}_to_{current_date_str}.html"
            fig.write_html(filename, include_plotlyjs='cdn', auto_open=False)
            print(f"Chart saved to {filename}")
    
    for choice in choices:
        start_date = get_start_date(current_date, choice)
        data = df.loc[start_date:current_date]
        title = f"All Funds Illustrate_B {choice} to {current_date_str}"
        fig = create_l_chart(data, ['Fund_A', 'Fund_B', 'Fund_C', 'Fund_D'], title, start_date, current_date, is_combined=True)
        filename = f"Combined_Illustrate_B_{choice}_to_{current_date_str}.html"
        fig.write_html(filename, include_plotlyjs='cdn', auto_open=False)
        print(f"Chart saved to {filename}")

# ───────────────────────────────────────────────────────────────
# MAIN INTERACTIVE PROGRAM
# ───────────────────────────────────────────────────────────────

print("Welcome to Fund Chart Generator")
action = input("Please choose your action: “Your_return” or “Past_return”, “Illustrate_return” or “Illustrate_B_return”: ").strip()

if action == "Your_return":
    fund = input("Please choose a fund: A,B,C or D: ").strip()
    start_date = input("Please input a start_date (YYYY-MM-DD): ").strip()
    current_date = input("Please input a current_date (YYYY-MM-DD): ").strip()
    your_return(fund, start_date, current_date)
elif action == "Past_return":
    fund = input("Please choose a fund: A,B,C or D: ").strip()
    end_date = input("Please input an end_date (YYYY-MM-DD): ").strip()
    choice = input("Please input one of: “Past_3_months”, “Past_12_months”, “Past_36_months”, “Past_60_months”, “Past_since_2016”, “Past_since_this_year”: ").strip()
    past_return(fund, end_date, choice)
elif action == "Illustrate_return":
    illustrate_return()
elif action == "Illustrate_B_return":
    illustrate_b_return()
else:
    print("Invalid action")

=== Loading and validating data ===
→ Loaded successfully!
Shape: (7672, 4)
Date range: 2015-12-31 → 2036-12-31

Key date availability:
   2015-12-31: Found → A=100.00  B=100.00  C=100.00  D=100.00
   2025-12-31: Found → A=761.45  B=282.69  C=1459.14  D=132.15
   2036-12-31: Found → A=1508.99  B=1044.09  C=1588.47  D=181.23

Data ready. Starting interactive mode...

Welcome to Fund Chart Generator
Chart saved to Fund_B_Your_return_2026-01-04_to_2029-09-09.html
